# Tensorflow的简单使用

https://www.tensorflow.org/tutorials/?hl=en

In [9]:
# 安装tensorflow GPU版本
!pip install tensorflow-gpu

    100% |████████████████████████████████| 258.8MB 46kB/s 
    100% |████████████████████████████████| 3.0MB 13.3MB/s 
    100% |████████████████████████████████| 51kB 14.0MB/s 
tensorflow 1.9.0 has requirement tensorboard<1.10.0,>=1.9.0, but you'll have tensorboard 1.11.0 which is incompatible.
keras 2.2.2 has requirement keras_applications==1.0.4, but you'll have keras-applications 1.0.5 which is incompatible.
keras 2.2.2 has requirement keras_preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.3 which is incompatible.
  Found existing installation: Keras-Preprocessing 1.0.2
    Uninstalling Keras-Preprocessing-1.0.2:
      Successfully uninstalled Keras-Preprocessing-1.0.2
  Found existing installation: tensorboard 1.9.0
    Uninstalling tensorboard-1.9.0:
      Successfully uninstalled tensorboard-1.9.0
  Found existing installation: Keras-Applications 1.0.4
    Uninstalling Keras-Applications-1.0.4:
      Successfully uninstalled Keras-Applications-1.0.4


In [10]:
# 仅使用1号GPU 参考 https://github.com/keras-team/keras/issues/6031
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
# 仅使用需要的显存
import tensorflow as tf
from tensorflow.keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

## 做手写数据库MNIST分类

In [12]:
# 读取数据
mnist = tf.keras.datasets.mnist
# 把数据分为训练数据和测试数据
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# 对数据进行归一化
x_train, x_test = x_train / 255.0, x_test / 255.0
# 可以看到训练数据60000枚，测试数据10000枚
(x_train.shape, x_test.shape)

((60000, 28, 28), (10000, 28, 28))

In [16]:
# 现在tensorflow集成了keras，可以很方便的创建神经网络

# 创建一个顺序模型（Sequential） https://keras.io/zh/models/sequential/
model = tf.keras.models.Sequential([
    # 输入层对图片进行展平，包含28*28=784个神经元 https://keras.io/zh/layers/core/#flatten
    tf.keras.layers.Flatten(),
    # 隐藏层：全连接层（Dense），包含512个神经元 https://keras.io/zh/layers/core/#dense
    # 使用 线性整流函数（ReLU）激活 https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E6%95%B4%E6%B5%81%E5%87%BD%E6%95%B0
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    # 增加一个20% Dropout防止过拟合 https://keras.io/zh/layers/core/#dropout
    tf.keras.layers.Dropout(0.2),
    # 输出层：全连接层，包含10个神经元，代表0-9十个数字，用softmax函数激活。
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# 编译网络
# 使用Adam优化器 https://keras.io/zh/optimizers/#adam
# 使用稀疏交叉熵（sparse_categorical_crossentropy）作为目的函数 https://keras.io/ja/objectives/
# 使用精度进行验证
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# 使用训练数据训练网络，一共训练5次（epoch），可以看到精度达到了98%以上
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.2208 - acc: 0.9346
Epoch 2/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.0955 - acc: 0.9708
Epoch 3/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.0695 - acc: 0.9779
Epoch 4/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.0530 - acc: 0.9831
Epoch 5/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.0431 - acc: 0.9862


In [18]:
# 使用测试数据对分类精度进行测试，在测试集上达到98.2%的精度
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 45us/step


[0.06517951938676415, 0.9809]

In [20]:
# 尝试再训练5组
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.0198 - acc: 0.9934
Epoch 2/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.0192 - acc: 0.9935
Epoch 3/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.0183 - acc: 0.9934
Epoch 4/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.0164 - acc: 0.9944
Epoch 5/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.0150 - acc: 0.9948


In [21]:
# 再测试，在测试数据提升不明显，训练数据明显，出现过拟合现象
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 27us/step


[0.07587860820080278, 0.9832]